# Intro to Recommender Systems Lab

Complete the exercises below to solidify your knowledge and understanding of recommender systems.

For this lab, we are going to be putting together a user similarity based recommender system in a step-by-step fashion. Our data set contains customer grocery purchases, and we will use similar purchase behavior to inform our recommender system. Our recommender system will generate 5 recommendations for each customer based on the purchases they have made.

In [247]:
import pandas as pd
from scipy.spatial.distance import pdist, squareform

In [248]:
data = pd.read_csv('../data/customer_product_sales.csv')

In [249]:
data.head()

,CustomerID,FirstName,LastName,SalesID,ProductID,ProductName,Quantity
0,61288,Rosa,Andersen,134196,229,Bread - Hot Dog Buns,16
1,77352,Myron,Murray,6167892,229,Bread - Hot Dog Buns,20
2,40094,Susan,Stevenson,5970885,229,Bread - Hot Dog Buns,11
3,23548,Tricia,Vincent,6426954,229,Bread - Hot Dog Buns,6
4,78981,Scott,Burch,819094,229,Bread - Hot Dog Buns,20


## Step 1: Create a data frame that contains the total quantity of each product purchased by each customer.

You will need to group by CustomerID and ProductName and then sum the Quantity field.

In [250]:
quantities = data.groupby(['CustomerID', 'ProductName'], as_index=False).agg({'Quantity':'sum'})
quantities.head()

,CustomerID,ProductName,Quantity
0,33,Apricots - Dried,1
1,33,Assorted Desserts,1
2,33,Bandage - Flexible Neon,1
3,33,"Bar Mix - Pina Colada, 355 Ml",1
4,33,"Beans - Kidney, Canned",1


## Step 2: Use the `pivot_table` method to create a product by customer matrix.

The rows of the matrix should represent the products, the columns should represent the customers, and the values should be the quantities of each product purchased by each customer. You will also need to replace nulls with zeros, which you can do using the `fillna` method.

In [251]:
quantities_pivot = pd.pivot_table(quantities, index='ProductName', columns='CustomerID', values='Quantity').fillna(0)
quantities_pivot.head()

CustomerID,33,200,264,356,412,464,477,639,649,669,...,97697,97753,97769,97793,97900,97928,98069,98159,98185,98200
ProductName,,,,,,,,,,,,,,,,,,,,,
Anchovy Paste - 56 G Tube,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,25.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
"Appetizer - Mini Egg Roll, Shrimp",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,25.0,25.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Appetizer - Mushroom Tart,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,25.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,25.0,0.0
Appetizer - Sausage Rolls,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,25.0,25.0,25.0,0.0,25.0,0.0
Apricots - Dried,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,25.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


## Step 3: Create a customer similarity matrix using `squareform` and `pdist`. For the distance metric, choose "euclidean."

In [252]:
distances = pd.DataFrame(1/(1 + squareform(pdist(quantities_pivot.T, 'euclidean'))), 
                         index=quantities_pivot.columns, columns=quantities_pivot.columns)

distances.head()

CustomerID,33,200,264,356,412,464,477,639,649,669,...,97697,97753,97769,97793,97900,97928,98069,98159,98185,98200
CustomerID,,,,,,,,,,,,,,,,,,,,,
33,1.000000,0.077421,0.087047,0.081800,0.080634,0.082709,0.074573,0.083020,0.081503,0.080070,...,0.004811,0.004669,0.004412,0.005019,0.004312,0.004515,0.004583,0.004355,0.004167,0.004333
200,0.077421,1.000000,0.078448,0.076435,0.073693,0.075255,0.075956,0.076435,0.077674,0.076923,...,0.004824,0.004681,0.004431,0.005047,0.004311,0.004521,0.004614,0.004367,0.004166,0.004335
264,0.087047,0.078448,1.000000,0.080070,0.081800,0.080350,0.076923,0.080634,0.082100,0.078448,...,0.004822,0.004674,0.004416,0.005035,0.004322,0.004543,0.004595,0.004365,0.004179,0.004333
356,0.081800,0.076435,0.080070,1.000000,0.076435,0.078187,0.075025,0.082403,0.077171,0.075956,...,0.004816,0.004671,0.004416,0.005038,0.004310,0.004526,0.004578,0.004365,0.004175,0.004339
412,0.080634,0.073693,0.081800,0.076435,1.000000,0.078711,0.075025,0.082403,0.078187,0.078448,...,0.004810,0.004702,0.004414,0.005034,0.004318,0.004530,0.004578,0.004367,0.004177,0.004349


## Step 4: Check your results by generating a list of the top 5 most similar customers for a specific CustomerID.

In [253]:
similarities = distances[33].sort_values(ascending=False)[1:]
some_values = []
for k, j in dict(similarities.head()).items():
    some_values.append(k)
some_values

[264, 3535, 3317, 2503, 3305]

## Step 5: From the data frame you created in Step 1, select the records for the list of similar CustomerIDs you obtained in Step 4.

In [254]:
data5 = quantities.loc[quantities['CustomerID'].isin(some_values)]
data5.head()

,CustomerID,ProductName,Quantity
131,264,Apricots - Halves,1
132,264,Apricots Fresh,1
133,264,Bacardi Breezer - Tropical,1
134,264,Bagel - Plain,1
135,264,Banana - Leaves,1


## Step 6: Aggregate those customer purchase records by ProductName, sum the Quantity field, and then rank them in descending order by quantity.

This will give you the total number of each product purchased by the 5 most similar customers to the customer you selected in order from most purchased to least.

In [255]:
data5_new = data5.groupby('ProductName', as_index=False).agg({'Quantity':'sum'}).sort_values(['Quantity'],ascending=False)
data5_new.head()

,ProductName,Quantity
33,Butter - Unsalted,3
203,Wine - Ej Gallo Sierra Valley,3
186,Towels - Paper / Kraft,3
168,Soup - Campbells Bean Medley,3
199,Wine - Blue Nun Qualitatswein,3


## Step 7: Filter the list for products that the chosen customer has not yet purchased and then recommend the top 5 products with the highest quantities that are left.

- Merge the ranked products data frame with the customer product matrix on the ProductName field.
- Filter for records where the chosen customer has not purchased the product.
- Show the top 5 results.

In [256]:
merged = pd.merge(data5_new, quantities_pivot, on='ProductName')
merged = merged.loc[merged[33]==0].sort_values(['Quantity'],ascending=False).head()
merged = pd.DataFrame(merged.ProductName)
merged

,ProductName
0,Butter - Unsalted
3,Soup - Campbells Bean Medley
4,Wine - Blue Nun Qualitatswein
1,Wine - Ej Gallo Sierra Valley
41,Beans - Wax


## Step 8: Now that we have generated product recommendations for a single user, put the pieces together and iterate over a list of all CustomerIDs.

- Create an empty dictionary that will hold the recommendations for all customers.
- Create a list of unique CustomerIDs to iterate over.
- Iterate over the customer list performing steps 4 through 7 for each and appending the results of each iteration to the dictionary you created.

In [284]:
empty = {}
CustomerIDs = [33, 200, 264, 356, 412, 464, 477, 639]
for i in CustomerIDs:
    similarities = distances[i].sort_values(ascending=False)[1:]
    some_values = []
    for k, j in dict(similarities.head()).items():
        some_values.append(k)
    data5 = quantities.loc[quantities['CustomerID'].isin(some_values)]
    data5_new = data5.groupby('ProductName', as_index=False).agg({'Quantity':'sum'}).sort_values(['Quantity'],ascending=False)
    merged = pd.merge(data5_new, quantities_pivot, on='ProductName')
    merged = merged.loc[merged[i]==0].sort_values(['Quantity'],ascending=False).head()
    merged = pd.DataFrame(merged.ProductName)
    empty[i] = merged
empty

{33:                       ProductName
 0                          Loquat
 12            Cheese Cloth No 100
 1                       Milk - 2%
 18                        Bananas
 16  Wine - Wyndham Estate Bin 777, 200:                        ProductName
 0  Muffin - Carrot Individual Wrap
 2           Beans - Kidney, Canned
 1                          Pomello
 4                     Berry Brulee
 6                    Pork - Kidney, 264:                           ProductName
 0                  Sauce - Demi Glace
 3          Wine - Two Oceans Cabernet
 1                 Flavouring - Orange
 11            Soupfoamcont12oz 112con
 18  Pasta - Cheese / Spinach Bauletti, 356:                ProductName
 0    Veal - Inside, Choice
 5   Tart Shells - Sweet, 4
 6     Lettuce - Spring Mix
 8      Beef - Inside Round
 9  Soupfoamcont12oz 112con, 412:              ProductName
 1          Cumin - Whole
 10  Lettuce - Spring Mix
 3             Watercress
 16           Blueberries
 14       Wonton W

In [277]:
lot_values = []
for k, j in dict(empty).items():
    lot_values.append(k)
lot_values

[33, 200, 264, 356, 412, 464, 477, 639]

##  Step 9: Store the results in a Pandas data frame. The data frame should a column for Customer ID and then a column for each of the 5 product recommendations for each customer.

In [268]:
for i in range(len(empty))
empty_df = pd.DataFrame.from_dict(empty, orient='index')
empty_df
for k, j in dict(empty).items():
    empty[33]

,0
33,ProductName 0 ...
200,ProductName 0 Soup ...
264,ProductName 0 ...
356,"ProductName 0 Veal - Inside, C..."
412,ProductName 0 Oliv...
464,ProductName 0 ...
477,ProductName 0 Oliv...
639,ProductName 0 Cod -...


## Step 10: Change the distance metric used in Step 3 to something other than euclidean (correlation, cityblock, consine, jaccard, etc.). Regenerate the recommendations for all customers and note the differences.

For Correlation

In [280]:
quantities = data.groupby(['CustomerID', 'ProductName'], as_index=False).agg({'Quantity':'sum'})
quantities_pivot = pd.pivot_table(quantities, index='ProductName', columns='CustomerID', values='Quantity').fillna(0)
distances = pd.DataFrame(1/(1 + squareform(pdist(quantities_pivot.T, 'jaccard'))), 
                         index=quantities_pivot.columns, columns=quantities_pivot.columns)
similarities = distances[33].sort_values(ascending=False)[1:]
some_values = []
for k, j in dict(similarities.head()).items():
    some_values.append(k)
data5 = quantities.loc[quantities['CustomerID'].isin(some_values)]
data5_new = data5.groupby('ProductName', as_index=False).agg({'Quantity':'sum'}).sort_values(['Quantity'],ascending=False)
merged = pd.merge(data5_new, quantities_pivot, on='ProductName')
merged = merged.loc[merged[33]==0].sort_values(['Quantity'],ascending=False).head()
merged = pd.DataFrame(merged.ProductName)
merged

,ProductName
0,Loquat
12,Cheese Cloth No 100
1,Milk - 2%
18,Bananas
16,Wine - Wyndham Estate Bin 777


For Correlation